In [ ]:
import numpy as np
from time import sleep
from astropy.io import fits
from astropy.time import Time

import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog

In [ ]:
def get_catalog(ra, dec, target, outpath, table='swiftmastr', oformat='fits'):

    # build the command
    cmd = (f'./browse_extract_wget.pl table={table} position={ra},{dec} '
           f'outfile={outpath}/{target}/{target}xrt.fits format={oformat} radius=10')

    print(cmd)
    os.system(cmd)

In [ ]:
def get_files(target, outpath):

    # at least one file was 0 bytes... 
    try:
        cat = fits.getdata(f'{outpath}/{target}/{target}xrt.fits')
    except OSError:
        return

    if cat.shape[0] < 1:
        return

    for i in range(cat.shape[0]):
        obsid = cat['obsid'][i]
        t = Time(cat['start_time'][i], format='mjd')
        y, m = t.iso.split('-')[:2]

        wget_base = (f'wget -P {outpath}/{target} -q -nH --no-check-certificate --cut-dirs=5 '
                '-r -l0 -c -nc -np -R "index*" -erobots=off --retr-symlinks '
                f'https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{y}_{m}/')
             
        for prod in ['xrt', 'auxil', 'log']:
            wget = f'{wget_base}{obsid}/{prod}/'
            os.system(wget)

In [ ]:
def write_info(outpath, idx, data):
    target = data.iloc[idx]['NAME'].replace(' ', '_')
    cols = data.columns.tolist()
    vals = data.iloc[idx]
    # convert the values into all strings
    vals_str = [f'{i:.8}' if isinstance(i, float) else i for i in vals.values.tolist()]
    with open(f'{outpath}/{target}/{target}.info', 'w') as f:
        f.write(f'{",".join(cols)}\n')
        f.write(f'{",".join(vals_str)}\n')        
    

In [ ]:
#%%capture

# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

for i, (ra, dec, name) in enumerate(zip(data['RA'], data['DEC'], data['NAME'])):

    #print(name)
    name = name.replace(' ', '_')

    if not os.path.isdir(f'{outpath}/{name}'):
        os.makedirs(f'{outpath}/{name}')

    #get_catalog(ra, dec, f'{name}', outpath)
    #write_info(outpath, i, data)
    get_files(f'{name}', outpath)
    
    sleep(1)

In [ ]:
wget -P {outpath}/{target} -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -nc -np -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/2011_05/00091167001/xrt/